In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

class ImageEditor:
    def __init__(self, root):
        self.root = root
        self.image_originale = None
        self.image_modifiee = None

        # Création de l'interface graphique
        self.creer_interface()

    def creer_interface(self):
        # Cadre pour l'image originale
        self.cadre_original = tk.Frame(self.root)
        self.cadre_original.pack(side=tk.LEFT, padx=10)

        # Cadre pour l'image modifiée
        self.cadre_modifie = tk.Frame(self.root)
        self.cadre_modifie.pack(side=tk.LEFT, padx=10)

        # Bouton pour charger une image
        self.bouton_charger = tk.Button(self.root, text="Charger Image", command=self.charger_image)
        self.bouton_charger.pack(pady=10)

        # Échelles pour les couleurs
        self.echelle_rouge = tk.Scale(self.root, from_=0, to=255, label="Rouge", orient=tk.HORIZONTAL, command=self.appliquer_modifications_auto)
        self.echelle_rouge.set(127)  # Initialiser à 50%
        self.echelle_rouge.pack(pady=5)

        self.echelle_vert = tk.Scale(self.root, from_=0, to=255, label="Vert", orient=tk.HORIZONTAL, command=self.appliquer_modifications_auto)
        self.echelle_vert.set(127)  # Initialiser à 50%
        self.echelle_vert.pack(pady=5)

        self.echelle_bleu = tk.Scale(self.root, from_=0, to=255, label="Bleu", orient=tk.HORIZONTAL, command=self.appliquer_modifications_auto)
        self.echelle_bleu.set(127)  # Initialiser à 50%
        self.echelle_bleu.pack(pady=5)

        # Bouton pour enregistrer l'image
        self.bouton_enregistrer = tk.Button(self.root, text="Enregistrer Image", command=self.enregistrer_image)
        self.bouton_enregistrer.pack(pady=10)

    def charger_image(self):
        chemin_image = filedialog.askopenfilename()
        if chemin_image:
            self.image_originale = Image.open(chemin_image)
            self.redimensionner_image()
            self.afficher_image_originale()

    def redimensionner_image(self):
        max_size = 500
        width, height = self.image_originale.size
        if width > max_size or height > max_size:
            ratio = min(max_size / width, max_size / height)
            new_width = int(width * ratio)
            new_height = int(height * ratio)
            self.image_originale = self.image_originale.resize((new_width, new_height), Image.LANCZOS)

    def afficher_image_originale(self):
        self.vider_cadre(self.cadre_original)
        photo = ImageTk.PhotoImage(self.image_originale)
        label = tk.Label(self.cadre_original, image=photo)
        label.image = photo
        label.pack()

    def afficher_image_modifiee(self, image):
        self.vider_cadre(self.cadre_modifie)
        photo = ImageTk.PhotoImage(image)
        label = tk.Label(self.cadre_modifie, image=photo)
        label.image = photo
        label.pack()

    def vider_cadre(self, cadre):
        for widget in cadre.winfo_children():
            widget.destroy()

    def appliquer_modifications(self):
        if self.image_originale:
            rouge = self.echelle_rouge.get()
            vert = self.echelle_vert.get()
            bleu = self.echelle_bleu.get()

            # Si les taux de rouge, vert et bleu sont tous égaux à 255, créer une image blanche
            if rouge == vert == bleu == 255:
                image_modifiee = Image.new("RGB", self.image_originale.size, (255, 255, 255))
            else:
                image_modifiee = Image.new("RGB", self.image_originale.size)
                pixels = self.image_originale.load()
                pixels_modifies = image_modifiee.load()

                for i in range(self.image_originale.size[0]):
                    for j in range(self.image_originale.size[1]):
                        pixel = pixels[i, j]
                        r, g, b = pixel[:3]  # Prendre les trois premières valeurs du tuple
                        pixels_modifies[i, j] = (int(r * (rouge / 255)), int(g * (vert / 255)), int(b * (bleu / 255)))

            self.afficher_image_modifiee(image_modifiee)
            self.image_modifiee = image_modifiee

    def appliquer_modifications_auto(self, *args):
        self.appliquer_modifications()

    def enregistrer_image(self):
        if self.image_modifiee:
            chemin_fichier = filedialog.asksaveasfilename(defaultextension=".png")
            if chemin_fichier:
                self.image_modifiee.save(chemin_fichier)

# Création de la fenêtre principale
root = tk.Tk()
root.title("Éditeur d'Images")
app = ImageEditor(root)
root.mainloop()
